In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import sklearn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.externals import joblib
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing 
from xgboost import XGBRegressor, plot_importance
from matplotlib import pyplot
import time

Using TensorFlow backend.


In [2]:
from datetime import datetime
parser = lambda date: pd.to_datetime(date, format='%d.%m.%Y')
train = pd.read_csv('processed_train.csv',parse_dates=['date'], date_parser=parser)
test  = pd.read_csv('processed_test.csv',parse_dates=['date'], date_parser=parser)

In [3]:
#features
df = train
df['day']=df['date'].apply(lambda x: x.strftime('%d'))
df['day']=df['day'].astype('int64')
df['month']=df['date'].apply(lambda x: x.strftime('%m'))
df['month']=df['month'].astype('int64')
df['year']=df['date'].apply(lambda x: x.strftime('%Y'))
df['year']=df['year'].astype('int64')
df = df[['day','month','year','item_id', 'shop_id','item_price','item_cnt_day']]
#     df = df.pivot_table(index=['item_id', 'shop_id'], columns='date', values='item_cnt_day', fill_value=0).reset_index()
#     count=df.iloc[:,2:]
#     df['total']=count.sum(axis=1)
data=df
#data = pd.merge(val, df, on=['item_id', 'shop_id'], how='left').fillna(0)
data['item_id'] = np.log1p(data['item_id'])

C:\Users\Siddu\Miniconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
train_date_info = data

In [5]:
y_train_normal = train_date_info['item_cnt_day']
x_train_normal = train_date_info.drop(labels=['item_cnt_day'], axis=1)

In [6]:
x_train_val = x_train_normal[-100:]
y_train_val = y_train_normal[-100:]

In [7]:
def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions - targets) ** 2))

In [8]:
def TreeRegressor_model(x_train, y_train):
    reg = ExtraTreesRegressor(n_estimators=520, max_depth=25,random_state=50)
    #reg = ExtraTreesRegressor()
    reg.fit(x_train, y_train)
    y_pre = reg.predict(x_train)
    score = np.sqrt(mean_squared_error(y_train, y_pre))
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip(0., 20.), y_pre.clip(0., 20.))))
    return reg

In [9]:
def light_gbm_model(x_train, y_train):
    lgb_params = {
        'feature_fraction': 1,
        'metric': 'rmse',
        'min_data_in_leaf': 16,
        'bagging_fraction': 0.85,
        'learning_rate': 0.03,
        'objective': 'mse',
        'bagging_seed': 2 ** 7,
        'num_leaves': 32,
        'bagging_freq': 3,
        'verbose': 0
    }
    estimator = lgb.train(lgb_params, lgb.Dataset(x_train, label=y_train), 300)
    y_pre = estimator.predict(x_train)
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip(0., 20.), y_pre.clip(0., 20.))))
    return estimator

In [10]:
def linear_model(x_train, y_train):
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    y_pre = lr.predict(x_train)
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip, y_pre.clip)))
    return lr

In [11]:
def xgb_model(x_train, y_train, x_train_val, y_train_val):
    model = XGBRegressor(
        max_depth=8,
        n_estimators=1000,
        min_child_weight=300,
        colsample_bytree=0.9,
        subsample=0.9,
        eta=0.15,
        seed=42)
    model.fit(
        x_train,
        y_train,
        eval_metric="rmse",
        eval_set=[(x_train, y_train), (x_train_val, y_train_val)],
        verbose=True,
        early_stopping_rounds=10)
    y_pre = model.predict(x_train)
    print('RMSE cliped:', np.sqrt(mean_squared_error(y_train.clip(0., 20.), y_pre.clip(0., 20.))))
    plot_importance(model)
    pyplot.show()
    return model

In [12]:
def pre_data(data_type, reg, x_test):
    if reg is None:
        reg = joblib.load('%s/%s_model_weight.model' % (out_path, data_type))
    y_pre = reg.predict(x_test)
    return y_pre

In [13]:
ts = time.time()
#xgb_model = xgb_model(x_train_normal[:-100], y_train_normal[:-100], x_train_val, y_train_val)
# linear_model = linear_model(x_train_normal, y_train_normal)
#light_gbm_model = light_gbm_model(x_train_normal, y_train_normal)
tree_model = TreeRegressor_model(x_train_normal, y_train_normal)
time.time() - ts

RMSE cliped: 0.4117675588900876


4753.8684186935425

In [14]:
test.head()

,date,date_block_num,shop_id,item_id,item_price,ID
0,2015-02-06,25,31,11208,699.0,2472142
1,2013-11-21,10,31,19525,149.0,1044690
2,2015-01-03,24,42,16169,299.0,2367559
3,2014-07-18,18,42,2737,199.0,1793501
4,2014-04-13,15,2,15229,1199.0,1585639


In [15]:
df = test
df['day']=df['date'].apply(lambda x:x.strftime('%d'))
df['day']=df['day'].astype('int64')
df['month']=df['date'].apply(lambda x: x.strftime('%m'))
df['month']=df['month'].astype('int64')
df['year']=df['date'].apply(lambda x: x.strftime('%Y'))
df['year']=df['year'].astype('int64')
df = df[['day','month','year','item_id', 'shop_id','item_price']]
data=df
#data = pd.merge(val, df, on=['item_id', 'shop_id'], how='left').fillna(0)
data['item_id'] = np.log1p(data['item_id'])
test_x = data
# test_x.columns = np.append(['shop_id', 'item_id'],np.arange(0, 36, 1))
test_y_1 = pre_data('normal', tree_model, test_x)
# test_y_2 = pre_data('light_gbm', light_gbm_model, test_x)
# test_y_3 = pre_data('linear', linear_model, test_x)
#test_y_4 = pre_data('xgb', xgb_model, test_x)
test_y = test_y_1
test.head()

C:\Users\Siddu\Miniconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,date,date_block_num,shop_id,item_id,item_price,ID,day,month,year
0,2015-02-06,25,31,11208,699.0,2472142,6,2,2015
1,2013-11-21,10,31,19525,149.0,1044690,21,11,2013
2,2015-01-03,24,42,16169,299.0,2367559,3,1,2015
3,2014-07-18,18,42,2737,199.0,1793501,18,7,2014
4,2014-04-13,15,2,15229,1199.0,1585639,13,4,2014


In [16]:
test['item_cnt_day'] = test_y
test[['ID', 'item_cnt_day']].to_csv('submission_Xtra regressor_520-25.csv', index=False)